In [25]:
!pip3 install tensorflow

The folder you are executing pip from can no longer be found.


In [26]:
import os
import random
import shutil
from kaggle.api.kaggle_api_extended import KaggleApi
import zipfile
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
import joblib

In [56]:
import os

face_data_dir = '../backend/kaggleData/real_and_fake_face'

if os.path.exists(face_data_dir):
    print(f"Contents of '{face_data_dir}':")
    for item in os.listdir(face_data_dir):
        item_path = os.path.join(face_data_dir, item)
        print(f"- {item}")
else:
    print(f"Error: Directory '{face_data_dir}' not found.")

Contents of '../backend/kaggleData/real_and_fake_face':
- training_fake
- training_real


In [ ]:
import os

face_data_dir = '../backend/kaggleData/real_and_fake_face_detection/real_and_fake_face'

if os.path.exists(face_data_dir):
    print(f"Contents of '{face_data_dir}':")
    for item in os.listdir(face_data_dir):
        item_path = os.path.join(face_data_dir, item)
        print(f"- {item}")
else:
    print(f"Error: Directory '{face_data_dir}' not found.")

Contents of '../backend/kaggleData/real_and_fake_face_detection/real_and_fake_face':
- training_fake
- training_real


In [67]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

base_dir = '../backend/kaggleData/real_and_fake_face' # The directory containing training_fake and training_real
real_dir = os.path.join(base_dir, 'training_real')
fake_dir = os.path.join(base_dir, 'training_fake')
img_height, img_width = 128, 128 # Choose your desired image size

def load_images_and_labels(real_dir, fake_dir, img_height, img_width):
    images = []
    labels = []
    for filename in os.listdir(real_dir):
        try:
            img_path = os.path.join(real_dir, filename)
            img = Image.open(img_path).resize((img_height, img_width))
            img_array = np.array(img) / 255.0
            images.append(img_array)
            labels.append(0) # 0 for real
        except Exception as e:
            print(f"Error loading real image {filename}: {e}")
    for filename in os.listdir(fake_dir):
        try:
            img_path = os.path.join(fake_dir, filename)
            img = Image.open(img_path).resize((img_height, img_width))
            img_array = np.array(img) / 255.0
            images.append(img_array)
            labels.append(1) # 1 for fake
        except Exception as e:
            print(f"Error loading fake image {filename}: {e}")
    return np.array(images), np.array(labels)

images, labels = load_images_and_labels(real_dir, fake_dir, img_height, img_width)
labels = to_categorical(labels, num_classes=2) # One-hot encode the labels

X_train, X_temp, y_train, y_temp = train_test_split(images, labels, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print(f"Total number of images: {len(images)}")
print(f"Training set size: {len(X_train)}")
print(f"Validation set size: {len(X_val)}")
print(f"Testing set size: {len(X_test)}")

Error loading real image real_00251.jpg: [Errno 2] No such file or directory
Error loading real image real_00537.jpg: [Errno 2] No such file or directory
Error loading real image real_00523.jpg: [Errno 2] No such file or directory
Error loading real image real_00245.jpg: [Errno 2] No such file or directory
Error loading real image real_00279.jpg: [Errno 2] No such file or directory
Error loading real image real_00292.jpg: [Errno 2] No such file or directory
Error loading real image real_00286.jpg: [Errno 2] No such file or directory
Error loading real image real_00735.jpg: [Errno 2] No such file or directory
Error loading real image real_00053.jpg: [Errno 2] No such file or directory
Error loading real image real_00047.jpg: [Errno 2] No such file or directory
Error loading real image real_00721.jpg: [Errno 2] No such file or directory
Error loading real image real_00709.jpg: [Errno 2] No such file or directory
Error loading real image real_00090.jpg: [Errno 2] No such file or directory

ValueError: With n_samples=0, test_size=0.3 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.